In [67]:
import pandas as pd 
import h5py 
import numpy as np 
import logging 

In [3]:
counts_genes_GTEX2019 = '/cluster/work/grlab/projects/GTEx/rna/results/splicing/spladder/genes_graph_conf2.merge_graphs.count.gene_expression.hdf5'
count_genes_GTEX2017 = '/cluster/work/grlab/projects/TCGA/PanCancer/rerun2018_alt_splice_GTEx/spladder/genes_graph_conf2.merge_graphs.validated.count.gene_expression.hdf5'

In [4]:
coding_genes_GTEX2017_annotv19 = '/cluster/work/grlab/projects/TCGA/PanCanAtlas/immunopepper_paper/peptides_ccell_rerun_gtex_151220/coding_genes_nocap_GTEX2017'
coding_genes_GTEX2019_annotv30 = '/cluster/work/grlab/projects/projects2020_OHSU/gene_lists/genes_coding_gencode_v30.txt'
coding_genes_GTEX2019_annotV32_intersectV30 = '/cluster/work/grlab/projects/projects2020_OHSU/gene_lists/genes_coding_gencode_v32_inter_v30.txt'

In [26]:
path_expr = counts_genes_GTEX2019
path_gene_list = coding_genes_GTEX2019_annotV32_intersectV30
path_save = '/cluster/work/grlab/projects/TCGA/PanCanAtlas/immunopepper_paper/peptides_ccell_rerun_gtex_151220/GTEX2017_commit_1fc5828_pya.0.17.1_ref'
tag = ".coding_gencode_v32_inter_v30.txt"
path_save = 'expression_counts.libsize{}.tsv'.format(tag)
percentile = 75 
h5_gene_expr = h5py.File(open(path_expr, 'rb'))
coding_list = pd.read_csv(path_gene_list, header = None, names = ['genes'])

In [57]:
h5_coding_mx = [list(h5_gene_expr["raw_count"][gene_id, 1:10])
                  for gene_id, gene_name in enumerate(h5_gene_expr['gene_ids'][1:100]) 
                  if gene_name.decode() in coding_list['genes'].values] 

In [58]:
list(h5_gene_expr["raw_count"][1, 1:10] )

[337733.0,
 231199.0,
 438070.0,
 229250.0,
 205955.0,
 484069.0,
 201409.0,
 277249.0,
 775853.0]

In [60]:
 np.array(h5_coding_mx).shape

(21, 9)

In [64]:
h5_coding_mx = np.array(h5_coding_mx)
#assert(h5_coding_mx.shape[0] == coding_list.shape[0])
sample_names = [strain.decode() for strain in h5_gene_expr['strains'][1:10]]
libsize_count= pd.DataFrame({'sample': sample_names ,
                                 'libsize_75percent': np.percentile(h5_coding_mx, percentile, axis = 0 )})

In [73]:
h5_gene_exr_reduce = h5_gene_expr

In [77]:
path_reduced = '/cluster/work/grlab/projects/tmp_laurie/genes_graph_conf2.merge_graphs.count.gene_expression.hdf5'
with h5py.File(open(path_expr, 'wb')) as f:
n_cls = 10
n_row = 100
h5_gene_exr_reduce['strains'] = h5_gene_exr_reduce['strains'][0:n_cls]
h5_gene_exr_reduce['gene_ids'] = h5_gene_exr_reduce['gene_ids'][0:n_row]
h5_gene_exr_reduce['raw_count'] = h5_gene_exr_reduce['raw_count'][0:n_row, 0:n_cls]

ValueError: Unable to create dataset (no write intent on file)

In [69]:
h5_gene_expr

<HDF5 file "genes_graph_conf2.merge_graphs.count.gene_expression.hdf5'>" (mode r)>

In [ ]:
def compute_library_size(path_expr, path_gene_list, path_save, percentile ):
    logging.info("Path of graph with gene expressions:  {}".format(path_expr))
    h5_gene_expr = h5py.File(open(path_expr, 'rb'))
    coding_list = pd.read_csv(path_gene_list, header = None, names = ['genes'])
    logging.info("Querying coding genes")
    h5_coding_mx = [list(h5_gene_expr["raw_count"][gene_id, :])
                  for gene_id, gene_name in enumerate(h5_gene_expr['gene_ids']) 
                  if gene_name.decode() in coding_list['genes'].values] 
    h5_coding_mx = np.array(h5_coding_mx)
    logging.info("Shape of coding matrix is: {}".format(h5_coding_mx.shape))
    logging.info("Number of coding genes is: {}".format(coding_list.shape))
    sample_names = [strain.decode() for strain in h5_gene_expr['strains']]
    logging.info("Computing library percentiles")
    libsize_count = pd.DataFrame({'sample': sample_names,
                                 'libsize_75percent': np.percentile(h5_coding_mx, percentile, axis = 0 )})
    logging.info("Saving")
    libsize_count.to_csv(path_save, index = None, sep = '\t')